In [18]:
import pandas as pd
import numpy as np
df  = pd.read_csv('covid_toy.csv')
df.sample(5)

,age,gender,fever,cough,city,has_covid
68,54,Female,104.0,Strong,Kolkata,No
40,49,Female,102.0,Mild,Delhi,No
18,64,Female,98.0,Mild,Bangalore,Yes
61,81,Female,98.0,Strong,Mumbai,No
29,34,Female,NaN,Strong,Mumbai,Yes


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,0:5],df.iloc[:,-1], test_size= 0.2)
X_test


,age,gender,fever,cough,city
43,22,Female,99.0,Mild,Bangalore
76,80,Male,100.0,Mild,Bangalore
67,65,Male,99.0,Mild,Bangalore
95,12,Female,104.0,Mild,Bangalore
35,82,Female,102.0,Strong,Bangalore
27,33,Female,102.0,Strong,Delhi
62,56,Female,104.0,Strong,Bangalore
14,51,Male,104.0,Mild,Bangalore
81,65,Male,99.0,Mild,Delhi
29,34,Female,NaN,Strong,Mumbai


In [13]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

##### Normally what we will do :

In [14]:
from sklearn.impute import SimpleImputer
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])
X_test_fever = si.fit_transform(X_test[['fever']])

In [15]:
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder(categories = [['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])
X_test_cough = oe.fit_transform(X_test[['cough']])

In [17]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(drop= 'first', sparse_output= False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

In [20]:
X_train_age = X_train.drop(columns= ['gender', 'fever', 'cough', 'city']).values
X_test_age = X_test.drop(columns= ['gender', 'fever', 'cough', 'city']).values


In [21]:
X_train_transformed = np.concatenate((X_train_age,X_train_cough,X_train_gender_city),axis= 1)
X_test_transformed = np.concatenate((X_test_age,X_test_cough,X_test_gender_city),axis= 1)
X_test_transformed

array([[22.,  0.,  0.,  0.,  0.,  0.],
       [80.,  0.,  1.,  0.,  0.,  0.],
       [65.,  0.,  1.,  0.,  0.,  0.],
       [12.,  0.,  0.,  0.,  0.,  0.],
       [82.,  1.,  0.,  0.,  0.,  0.],
       [33.,  1.,  0.,  1.,  0.,  0.],
       [56.,  1.,  0.,  0.,  0.,  0.],
       [51.,  0.,  1.,  0.,  0.,  0.],
       [65.,  0.,  1.,  1.,  0.,  0.],
       [34.,  1.,  0.,  0.,  0.,  1.],
       [13.,  1.,  0.,  0.,  1.,  0.],
       [34.,  1.,  0.,  1.,  0.,  0.],
       [14.,  1.,  1.,  0.,  0.,  0.],
       [19.,  0.,  0.,  0.,  1.,  0.],
       [11.,  0.,  1.,  0.,  0.,  0.],
       [72.,  0.,  1.,  0.,  0.,  0.],
       [84.,  0.,  0.,  0.,  0.,  0.],
       [ 6.,  0.,  0.,  0.,  1.,  0.],
       [73.,  0.,  0.,  1.,  0.,  0.],
       [20.,  1.,  1.,  1.,  0.,  0.]])

##### What we should actually do:

In [25]:
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer(transformers= [
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories= [['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(drop = 'first',sparse_output= False),['gender','city'])
], remainder= 'passthrough')
ct.fit_transform(X_train)
ct.fit_transform(X_test)


array([[ 99.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  22.        ],
       [100.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  80.        ],
       [ 99.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  65.        ],
       [104.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  12.        ],
       [102.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  82.        ],
       [102.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  33.        ],
       [104.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  56.        ],
       [104.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  51.        ],
       [ 99.        ,   0.        ,   1.        ,   1.        ,
          0.    